In [9]:
%%script false --no-raise-error
import json
from google.colab import userdata
from google.oauth2 import service_account
from google.cloud.bigquery import magics

credentials_json = userdata.get('BIGQUERY_CREDENTIALS')
credentials = service_account.Credentials.from_service_account_info(json.loads(credentials_json))
magics.context.credentials = credentials

Couldn't find program: 'false'


In [ ]:
from google.cloud import bigquery
from google.cloud.bigquery import magics
%load_ext bigquery_magics

data_set = "testing_set"
project_name = "emerald-entity-468916-f9"
library_path = "gdelt-bq.internetarchivebooks"

job_config = bigquery.QueryJobConfig(default_dataset = f"{project_name}.{data_set}")
client = bigquery.Client(project = project_name, default_query_job_config = job_config, credentials = globals().get('credentials', None))
magics.context.default_query_job_config = job_config
magics.context.project = project_name

number_of_books = 5
start_year = 1800 # min 1800
end_year = 1922 # max 1922
max_book_length = 500000
min_book_length = 200000

The bigquery_magics extension is already loaded. To reload it, use:
  %reload_ext bigquery_magics


In [11]:
%%bigquery
CREATE OR REPLACE TABLE tmp_books_sample(GKGRECORDID STRING, DATE INTEGER, BookMeta_Title STRING, V2Themes STRING, V2Persons STRING, BookMeta_FullText STRING, rank INT64);

Query is running:   0%|          |

""


In [12]:
import random

years_sample = min(number_of_books, end_year - start_year + 1)
years = random.sample(range(start_year, end_year + 1), years_sample)
limit = int(number_of_books / years_sample * 10)

add_to_samples = """INSERT INTO tmp_books_sample(GKGRECORDID, DATE, BookMeta_Title, V2Themes, V2Persons, BookMeta_FullText)
SELECT * FROM (
SELECT GKGRECORDID, DATE, BookMeta_Title, V2Themes, V2Persons, BookMeta_FullText FROM {library_path}.{year} sb
TABLESAMPLE SYSTEM (2 PERCENT) WHERE length(BookMeta_FullText) <= {max_book_length} AND length(BookMeta_FullText) >= {min_book_length} LIMIT {limit}) s
WHERE NOT EXISTS (SELECT 1 FROM tmp_books_sample s1 WHERE s.GKGRECORDID = s1.GKGRECORDID)"""

for year in years:
    fill_table_sql = add_to_samples.format(year = year, library_path = library_path, max_book_length = max_book_length, min_book_length = min_book_length, limit = limit)
    client.query_and_wait(fill_table_sql)

NotFound: 404 POST https://bigquery.googleapis.com/bigquery/v2/projects/emerald-entity-468916-f9/queries?prettyPrint=false: Not found: Table gdelt-bq:internetarchivebooks.1686 was not found in location US

In [ ]:
%%bigquery --pyformat

UPDATE tmp_books_sample SET rank = 
    CAST(AI.GENERATE(FORMAT(p.prompt, TO_JSON_STRING(STRUCT(DATE as relase_date, BookMeta_Title as title, V2Themes as themes, LPAD(BookMeta_FullText, 5000) as beginning))),
    connection_id => 'us.ai_connection', endpoint => 'gemini-2.5-flash-lite').result AS INT64)
FROM prompts p WHERE p.code = 'rank_books';

INSERT INTO books (book_id, date, title, original_txt)
select GKGRECORDID, DATE, BookMeta_Title, BookMeta_FullText from tmp_books_sample order by rank desc, rand() limit {number_of_books};

Query is running:   0%|          |

""
